## DCGAN

Resource Link: https://youtu.be/IZtv9s_Wx9I

Paper Link: https://arxiv.org/abs/1511.06434

In [29]:
# Clear previous tensorboard logs
%rm -rf logs/

In [30]:
print("DCGAN - Deep Convolution GAN")

DCGAN - Deep Convolution GAN


In [31]:
%reload_ext tensorboard

In [32]:
# Start the tensorboard server (default is port 6006)
%tensorboard --logdir=logs/ --reload_interval=5

Reusing TensorBoard on port 6006 (pid 16970), started 0:14:48 ago. (Use '!kill 16970' to kill it.)

In [33]:
import torch
import torch.nn as nn


In [34]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # Input: N x channels_img x 64 x 64
            nn.Conv2d(channels_img, features_d, kernel_size=4, stride=2, padding=1),    # 32x32
            nn.LeakyReLU(0.2),
            self._block(features_d, features_d * 2, 4,2,1),     # 16x16
            self._block(features_d * 2, features_d * 4, 4,2,1), # 8x8
            self._block(features_d*4, features_d * 8, 4,2,1),   # 4x4
            nn.Conv2d(features_d*8, 1, kernel_size=4, stride=2, padding=0),
            nn.Sigmoid(),   # to ensure that the valuea are between 0 and 1
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False,),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self,x):
        return self.disc(x)
        

In [35]:

class Generator(nn.Module):
    def __init__(self, z_dim, channels_img, features_g):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            # Input: N x z_dim x 1 x 1
            self._block(z_dim, features_g*16, 4,1,0),   # N x f_g*16 x 4 x 4
            self._block(features_g*16, features_g*8, 4,2,1),    # 8x8
            self._block(features_g*8, features_g*4, 4,2,1),     # 16x16
            self._block(features_g*4, features_g*2, 4,2,1),     # 32x32
            nn.ConvTranspose2d(
                features_g*2, channels_img, kernel_size=4, stride=2, padding=1,
            ),      # 64x64
            nn.Tanh(),  # [-1,1]  
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels, out_channels, kernel_size, stride, padding, bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.gen(x)

In [36]:
# Mean=0, standatd deviaiton = 0.02
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)
            

In [37]:
def test():
    N, in_channels, H, W = 8,3,64,64
    z_dim = 100
    x = torch.randn((N, in_channels, H, W))
    disc = Discriminator(in_channels, 8)
    initialize_weights(disc)
    assert disc(x).shape == (N,1,1,1)
    
    gen = Generator(z_dim, in_channels, 8)
    z = torch.randn((N, z_dim, 1, 1))
    initialize_weights(gen)
    
    assert gen(z).shape == (N, in_channels, H, W)
    print("Success")

In [38]:
test()

Success


### Setup of Training

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
# from model import Discriminator, Generator, initialize_weights
 

Setting up the Hyperparameters ,etc.


In [40]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
LEARNING_RATE = 2e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
CHANNELS_IMG = 1
Z_DIM = 100
NUM_EPOCHS = 10
FEATURES_DISC = 64
FEATURES_GEN = 64

In [41]:
transforms = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.5 for _ in range(CHANNELS_IMG)], [0.5 for _ in range(CHANNELS_IMG)]
        ),
    ]
)

In [42]:
dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms, download=True)

loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

gen = Generator(Z_DIM, CHANNELS_IMG, FEATURES_GEN).to(device)
disc = Discriminator(CHANNELS_IMG, FEATURES_DISC).to(device)

initialize_weights(gen)
initialize_weights(disc)

opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5,0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5,0.999))
criterion = nn.BCELoss()

fixed_noise = torch.randn(32, Z_DIM, 1,1).to(device)
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")

step = 0

## Send both the networks to training modes
print(gen.train())
disc.train()

Generator(
  (gen): Sequential(
    (0): Sequential(
      (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): Sequential(
      (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (4): ConvTranspose2d(128, 1, 

Discriminator(
  (disc): Sequential(
    (0): Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Sequential(
      (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (5): Conv2d(512, 1, kernel_size=(4, 4), stride=(2, 2))
    (6): Sigmoid()
  )
)

In [43]:
for epoch in range(NUM_EPOCHS):
    for batch_idx, (real, _) in enumerate(loader):
        real = real.to(device)
        noise = torch.randn((BATCH_SIZE, Z_DIM, 1,1)).to(device)
        fake = gen(noise)

        ### Train the discriminator: max log(D(x)) + log(1-D(G(x)))
        disc_real = disc(real).reshape(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))

        loss_disc = (loss_disc_real + loss_disc_fake)/2
        disc.zero_grad()
        loss_disc.backward(retain_graph = True)
        opt_disc.step()

        ### Train the Generator: min log(1-D(G(z))) <-> max log(D(G(z)))
        output = disc(fake).reshape(-1)
        loss_gen = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        ###  Print Losses occasionally and print to tensorboard
        if batch_idx % 100 == 0:
            print(f"Epoch [{epoch}/{NUM_EPOCHS} Batch {batch_idx}/{len(loader)} \n Loss D: {loss_disc:.4f}, Loss G: {loss_gen:.4f}]")

            with torch.no_grad():
                fake = gen(fixed_noise)
                # Take out (upto) 32 examples
                img_grid_real = torchvision.utils.make_grid(
                    real[:32], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True
                )

                writer_real.add_image("Real", img_grid_real, global_step=step)

                writer_fake.add_image("Fake", img_grid_fake, global_step=step)
            
            step += 1


Epoch [0/10 Batch 0/469 
 Loss D: 0.6924, Loss G: 0.7892]
Epoch [0/10 Batch 100/469 
 Loss D: 0.0149, Loss G: 4.1057]
Epoch [0/10 Batch 200/469 
 Loss D: 1.1873, Loss G: 0.2146]
Epoch [0/10 Batch 300/469 
 Loss D: 0.6181, Loss G: 1.8031]
Epoch [0/10 Batch 400/469 
 Loss D: 0.4916, Loss G: 1.4346]
Epoch [1/10 Batch 0/469 
 Loss D: 0.5818, Loss G: 1.3469]
Epoch [1/10 Batch 100/469 
 Loss D: 0.6230, Loss G: 0.5844]
Epoch [1/10 Batch 200/469 
 Loss D: 0.6292, Loss G: 0.9937]
Epoch [1/10 Batch 300/469 
 Loss D: 0.5880, Loss G: 0.9252]
Epoch [1/10 Batch 400/469 
 Loss D: 0.5647, Loss G: 1.2622]
Epoch [2/10 Batch 0/469 
 Loss D: 0.6065, Loss G: 1.1093]
Epoch [2/10 Batch 100/469 
 Loss D: 0.5863, Loss G: 0.9617]
Epoch [2/10 Batch 200/469 
 Loss D: 0.5826, Loss G: 0.9707]
Epoch [2/10 Batch 300/469 
 Loss D: 0.6570, Loss G: 0.6381]
Epoch [2/10 Batch 400/469 
 Loss D: 0.6165, Loss G: 0.8440]
Epoch [3/10 Batch 0/469 
 Loss D: 0.5902, Loss G: 0.9959]
Epoch [3/10 Batch 100/469 
 Loss D: 0.5118, Loss

In [44]:
%tensorboard --logdir=logs/ --reload_interval=5

Reusing TensorBoard on port 6006 (pid 16970), started 0:37:37 ago. (Use '!kill 16970' to kill it.)